<a href="https://colab.research.google.com/github/Bhardwaj-Saurabh/Agile-Development-with-Azure/blob/master/ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai groq colorama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.6 MB/s eta 0:00:00


In [1]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [292]:
from IPython.display import display, HTML

def safe_colored(text, color):
    # HTML for Colab
    html = f"<span style='color:{color}'>{text}</span>"
    display(HTML(html))

safe_colored("Hello in blue", "red")
safe_colored("Warning in yellow", "orange")

In [12]:
from typing import Callable
class Tool:
    """
    A class representing a reusable piece of code (Tool).

    Attributes:
        name (str): Name of the tool.
        description (str): A textual description of what the tool does.
        func (callable): The function this tool wraps.
        arguments (list): A list of argument.
        outputs (str or list): The return type(s) of the wrapped function.
    """
    def __init__(self,
                 name: str,
                 description: str,
                 func: Callable,
                 arguments: list,
                 outputs: str):
        self.name = name
        self.description = description
        self.func = func
        self.arguments = arguments
        self.outputs = outputs

    def to_string(self) -> str:
        """
        Return a string representation of the tool,
        including its name, description, arguments, and outputs.
        """
        args_str = ", ".join([
            f"{arg_name}: {arg_type}" for arg_name, arg_type in self.arguments
        ])

        return (
            f"Tool Name: {self.name},"
            f" Description: {self.description},"
            f" Arguments: {args_str},"
            f" Outputs: {self.outputs}"
        )

    def __call__(self, *args, **kwargs):
        """
        Invoke the underlying function (callable) with provided arguments.
        """
        return self.func(*args, **kwargs)

In [13]:
import inspect

def tool(func):
    """
    A decorator that creates a Tool instance from the given function.
    """
    # Get the function signature
    signature = inspect.signature(func)

    # Extract (param_name, param_annotation) pairs for inputs
    arguments = []
    for param in signature.parameters.values():
        annotation_name = (
            param.annotation.__name__
            if hasattr(param.annotation, '__name__')
            else str(param.annotation)
        )
        arguments.append((param.name, annotation_name))

    # Determine the return annotation
    return_annotation = signature.return_annotation
    if return_annotation is inspect._empty:
        outputs = "No return annotation"
    else:
        outputs = (
            return_annotation.__name__
            if hasattr(return_annotation, '__name__')
            else str(return_annotation)
        )

    # Use the function's docstring as the description (default if None)
    description = func.__doc__ or "No description provided."

    # The function name becomes the Tool name
    name = func.__name__

    # Return a new Tool instance
    return Tool(
        name=name,
        description=description,
        func=func,
        arguments=arguments,
        outputs=outputs
    )

In [132]:
import json

@tool
def calculator(a: float,
               b: float,
               operation: str) -> dict:
    """
    Calculator to perform mathamatical operation between two numbers.

    operation:
        - add: for addition
        - multiply: for multiplication
        - subtract: for subtraction
        - divide: for division
    """
    if operation == 'add':
        return {"tool_answer": a + b}
    elif operation == 'multiply':
        return {"tool_answer": a * b}
    elif operation == 'subtract':
        return {"tool_answer": a - b}
    elif operation == 'divide':
        return {"tool_answer": a / b}
    else:
        return {"tool_answer": None}

In [133]:
tool_definition = calculator.to_string()
print(tool_definition)

Tool Name: calculator, Description: 
    Calculator to perform mathamatical operation between two numbers.

    operation: 
        - add: for addition
        - multiply: for multiplication
        - subtract: for subtraction
        - divide: for division
    , Arguments: a: float, b: float, operation: str, Outputs: dict


In [218]:
from openai import OpenAI

In [206]:
OPENAI_CLIENT = OpenAI()

In [244]:
ReAct_SYSTEM_PROMPT = """Act as a helpful assistant.
You run in a loop of Thought, Action, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Repeat till you get to the answer for the given user query.

Tools at your disposal to perform tasks as needed:
- %s

### IMPORTANT:
Do not solve the problem yourself.
Only output your next step.

If tool is needed. Return the following:
<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

You will be provided with the tool_call result as follows.
<observation>
{"tool_answer": answer}
</observation>

Use observation and proceed with the remaining steps.


Example session:

<question> add 2 and 3 </question>
<thought>I need to add 2 and 3 </thought>
<tool_call>{"name": "calculator","arguments": {"a": 1234, "b": 5678, "operation": "add"}}</tool_call>

You will be called again with this:

<observation>{1: {'tool_answer': 5}}</observation>

You then output:

<response>The final answer to your query is 5</response>
"""

In [245]:
ReAct_SYSTEM_PROMPT = ReAct_SYSTEM_PROMPT % tool_definition
print(ReAct_SYSTEM_PROMPT)

Act as a helpful assistant.
You run in a loop of Thought, Action, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Repeat till you get to the answer for the given user query.

Tools at your disposal to perform tasks as needed:
- Tool Name: calculator, Description: 
    Calculator to perform mathamatical operation between two numbers.

    operation: 
        - add: for addition
        - multiply: for multiplication
        - subtract: for subtraction
        - divide: for division
    , Arguments: a: float, b: float, operation: str, Outputs: dict

### IMPORTANT: 
Do not solve the problem yourself. 
Only output your next step. 

If tool is needed. Return the following:
<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

You will be provided with the tool_call resul

In [269]:
user_query = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, devide it by 34"
chat_history = [
    {
        "role": "system",
        "content": ReAct_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{user_query}</question>"
    }
]

In [270]:
response = OPENAI_CLIENT.chat.completions.create(
    messages=chat_history,
    model="gpt-4o-mini"
).choices[0].message.content

safe_colored(response, 'orange')

In [271]:
if 'tool_call' in response:
    print(True)

True


In [272]:
import re
from dataclasses import dataclass


@dataclass
class ToolContent:
    """class to represent the result of extracting tag content."""
    content: list[str]
    found: bool

def extract_llm_call(text: str, tag: str) -> ToolContent:
    """
    Extracts all content enclosed by specified tags (e.g., <thought>, <response>, etc.).
    """
    # Build the regex pattern dynamically to find multiple occurrences of the tag
    tag_pattern = rf"<{tag}>(.*?)</{tag}>"

    # Use findall to capture all content between the specified tag
    matched_contents = re.findall(tag_pattern, text, re.DOTALL)

    # Return the dataclass instance with the result
    return ToolContent(
        content=[content.strip() for content in matched_contents],
        found=bool(matched_contents),
    )

In [273]:
tool_call = extract_llm_call(response, 'tool_call')

In [274]:
safe_colored(tool_call, 'yellow')

In [275]:
tool_call = json.loads(tool_call.content[0])
safe_colored(tool_call, 'yellow')

In [276]:
tool_result = calculator(**tool_call['arguments'])
safe_colored(tool_result, 'skyblue')

In [288]:

user_query = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, devide it by 34"

chat_history = [
    {
        "role": "system",
        "content": ReAct_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{user_query}</question>"
    }
]

In [289]:
while True:
    response = OPENAI_CLIENT.chat.completions.create(
                                messages=chat_history,
                                model="gpt-4o-mini"
                                ).choices[0].message.content

    safe_colored(response, 'orange')
    print()
    if 'tool_call' in response:
        tool_call = extract_llm_call(response, tag="tool_call")
        tool_call = json.loads(tool_call.content[0])
        result = calculator(**tool_call['arguments'])
        safe_colored(result, 'skyblue')
        chat_history.append(
                            {
                                "role": "user",
                                "content": f"<observation>{result}</observation>"
                            }
                        )

    if 'response' in response:
        print()
        safe_colored(response, 'green')
        break

In [290]:
final_answer = extract_llm_call(response, tag="response")
print(final_answer.content[0])

The final answer to your query is approximately 1016.47
